-   This code snippet calculates the negative log-likelihoods and embeddings for the most likely and second most likely generations in a given set of sequences. 
-   It uses a pre-trained model to compute these metrics, considering the prompt and target IDs, and stores the results for further analysis.

In [2]:
import os
import random
import numpy as np
import torch

# Please make sure you are using CUDA enabled GPU for this project
device = 'cuda'

# Setting the seed value ensures that the results are reproducible across different runs
seed_val = 10

# Ensuring that the seed is set for Python's hashing, random operations, NumPy, and PyTorch
os.environ['PYTHONHASHSEED'] = str(seed_val)
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)

In [3]:
# LLM Parameter Tuning
params = {    
    # Model related arguments
    'evaluation_version': 'opt-125m',
    'generation_version': 'opt-125m',
    'experiment_id': 'run_1',
}

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

evaluation_model = params['evaluation_version']

# Getting the model from params and loading it to the GPU
# Since we will be using the same model for other notebooks, we will save it in the cache directory
model = AutoModelForCausalLM.from_pretrained(f"facebook/{evaluation_model}", torch_dtype=torch.float16, cache_dir='./cache_dir').cuda()

tokenizer = AutoTokenizer.from_pretrained(f"facebook/{evaluation_model}", use_fast=False, cache_dir='./cache_dir')

c:\Users\dhair\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\content\drive\MyDrive\NLP Proj\hf_cache_dir. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [6]:
import wandb


# We are using wandb to track our experiments
wandb.init(project='nlg_uncertainty', id=params['experiment_id'], config=params, resume='allow')

run_version = wandb.run.name

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 4data692 (nlp53113). Use `wandb login --relogin` to force relogin


In [7]:
import pickle

generation_model = params['generation_version']

# Please generate the responses using the generation notebook first
# Please run the cleaner notebook before running this notebook
# Load the responses for the given run and model version
with open(f'./sequences/{run_version}/{generation_model}_cleaned_generations.pkl', 'rb') as infile:
    responses = pickle.load(infile)

# Load the semantic similarities for the given run and model version
with open(f'./ss/{run_version}/{generation_model}_generations_similarities.pkl', 'rb') as infile:
    similarities_dict = pickle.load(infile)

In [39]:
# Checking sample response of the model
for sample in responses:
    print(sample)
    print("")
    break

{'prompt': tensor([    2, 10554, 25596,    16,    41,   470,  1612,   433,  3468,  2164,
           30,  3421,   603,     4,  3139,  1403,    12,    90, 22764,  4320,
           34,    81,   155,   153,  8291,     8,    16,  1166,    30,   883,
          153,    82,   349,   186,     6,   217,    81,   504,   153,   604,
            4,    85,    21,     5,    78,  4320,    19, 17268,    81,    65,
          153,     7,   339,     5,   496, 10202,  3683,    13,  1292, 19092,
         2330,     4,  3139,  6966, 18605,   696,     6,    61,    34,    57,
         1027,   187, 17616,     6,    16,   122,    41,  1013, 10467,   515,
           14, 17382,    63,   308,  2384,   924,     6,  3424,     8, 32895,
            4,  1437, 50118, 50118,   970,    58,    80, 15829,  1440,    22,
        10554, 25596,   113,   137,     5,   595,  4320,   880,    15,   830,
          545,     6, 24712,     4,    96, 31025,     6, 10125,  9555,  2543,
          523,  1412,    22, 10554, 25596,   113,    

In [8]:
def compute_neg_log_likelihoods(model, responses):
    """This function computes various negative log-likelihoods and other metrics for a given set of responses."""

    with torch.no_grad():
        result = []
        for sample in responses:
            
            result_dict = {}
            prompt = sample['prompt']
            generations = sample['cleaned_generations'].to(device) if 'cleaned_generations' in sample else sample['generations'].to(device)
            id_ = sample['id']
            
            # Initialize tensors to store various metrics
            average_neg_log_likelihoods, average_unconditioned_neg_log_likelihoods, neg_log_likelihoods, neg_unconditioned_log_likelihoods, pointwise_mutual_information = torch.zeros((generations.shape[0],)), torch.zeros((generations.shape[0],)), torch.zeros((generations.shape[0],)), torch.zeros((generations.shape[0],)), torch.zeros((generations.shape[0],))
            sequence_embeddings = []

            # Iterating through each generation and compute metrics
            for generation_index in range(generations.shape[0]):
                
                prompt = prompt[prompt != tokenizer.pad_token_id]
                generation = generations[generation_index][generations[generation_index] != tokenizer.pad_token_id]

                # This computation of the negative log likelihoods follows this tutorial: https://huggingface.co/docs/transformers/perplexity
                # Compute the negative log likelihoods following the Hugging Face tutorial
                target_ids = generation.clone()
                target_ids[:len(prompt)] = -100
                model_output = model(torch.reshape(generation, (1, -1)), labels=target_ids, output_hidden_states=True)
                generation_only = generation.clone()[(len(prompt) - 1):]
                unconditioned_model_output = model(torch.reshape(generation_only, (1, -1)), labels=generation_only, output_hidden_states=True)
                hidden_states = model_output['hidden_states']
                average_neg_log_likelihood = model_output['loss']

                # Compute various likelihoods and information metrics
                average_unconditioned_neg_log_likelihood = unconditioned_model_output['loss']
                average_neg_log_likelihoods[generation_index] = average_neg_log_likelihood
                average_unconditioned_neg_log_likelihoods[generation_index] = average_unconditioned_neg_log_likelihood
                neg_log_likelihoods[generation_index] = average_neg_log_likelihood * (len(generation) - len(prompt))
                neg_unconditioned_log_likelihoods[generation_index] = average_unconditioned_neg_log_likelihood * (
                    len(generation) - len(prompt))
                pointwise_mutual_information[generation_index] = -neg_log_likelihoods[
                    generation_index] + neg_unconditioned_log_likelihoods[generation_index]

                # Compute the average of the last layer's token embeddings
                average_of_last_layer_token_embeddings = torch.mean(hidden_states[-1], dim=1)
                sequence_embeddings.append(average_of_last_layer_token_embeddings)

            # Compute metrics for the most likely generations
            most_likely_generation = sample['most_likely_generation_ids'].to(device)
            target_ids = most_likely_generation.clone()
            target_ids[:len(prompt)] = -100
            model_output = model(torch.reshape(most_likely_generation, (1, -1)), labels=target_ids, output_hidden_states=True)
            hidden_states = model_output['hidden_states']
            average_neg_log_likelihood_of_most_likely_gen = model_output['loss']
            most_likely_generation_embedding = torch.mean(hidden_states[-1], dim=1)

            # Compute metrics for the second most likely generation
            second_most_likely_generation = sample['second_most_likely_generation_ids'].to(device)
            target_ids = second_most_likely_generation.clone()
            target_ids[:len(prompt)] = -100
            model_output = model(torch.reshape(second_most_likely_generation, (1, -1)), labels=target_ids, output_hidden_states=True)
            hidden_states = model_output['hidden_states']
            average_neg_log_likelihood_of_second_most_likely_gen = model_output['loss']

            neg_log_likelihood_of_most_likely_gen = average_neg_log_likelihood_of_most_likely_gen * (len(most_likely_generation) - len(prompt))

            sequence_embeddings = torch.stack(sequence_embeddings)
            
            # Compile all the computed metrics into a dictionary
            result_dict.update({
                'prompt': prompt,
                'generations': generations,
                'average_neg_log_likelihoods': average_neg_log_likelihoods,
                'neg_log_likelihoods': neg_log_likelihoods,
                'sequence_embeddings': most_likely_generation_embedding,
                'most_likely_sequence_embedding': most_likely_generation,
                'average_unconditioned_neg_log_likelihoods': average_unconditioned_neg_log_likelihoods,
                'neg_unconditioned_log_likelihoods': neg_unconditioned_log_likelihoods,
                'pointwise_mutual_information': pointwise_mutual_information,
                'average_neg_log_likelihood_of_most_likely_gen': average_neg_log_likelihood_of_most_likely_gen,
                'average_neg_log_likelihood_of_second_most_likely_gen': average_neg_log_likelihood_of_second_most_likely_gen,
                'neg_log_likelihood_of_most_likely_gen': neg_log_likelihood_of_most_likely_gen,
                'semantic_set_ids': torch.tensor(similarities_dict[id_[0]]['semantic_set_ids'], device=device),
                'id': id_
            })
            results.append(result_dict)

        return result

In [15]:
# Please use the models below you have some free memory in your GPU
# If you don't have enough memory, then it can lead to a crash (will require restart of the kernel/system)
torch.cuda.mem_get_info()

(4694474752, 6441926656)

In [14]:
# Collect garbage to free up CUDA memory
import gc
gc.collect()

torch.cuda.empty_cache()

In [11]:
likelihoods = get_neg_loglikelihoods(model, sequences)

In [45]:
import pathlib

pathlib.Path(f'./log/' + run_name).mkdir(parents=True, exist_ok=True)

In [12]:
with open(f'./log/{run_version}/{generation_version}_generations_{args.evaluation_version}_likelihoods.pkl','wb') as outfile:
    pickle.dump(likelihoods, outfile)